<a href="https://colab.research.google.com/github/wirt30435/travel_product_matching/blob/en_mapping/%E6%AF%94%E5%B0%8D%E6%97%85%E9%81%8A%E7%94%A2%E5%93%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [1]:
# 📌 安裝套件
!pip install --upgrade google-cloud-bigquery
!pip install pandas jieba rapidfuzz googletrans==4.0.0-rc1

# 📌 匯入套件
import pandas as pd
import jieba
from google.colab import auth
from google.cloud import bigquery
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from datetime import datetime
from googletrans import Translator
import itertools
import os

# ✅ Step 1: Google 認證
auth.authenticate_user()

# ✅ Step 2: BigQuery client 初始化
project_id = "bishare-1606"
dataset = "jessica_test"
client = bigquery.Client(project=project_id)

# ✅ Step 3: 翻譯功能（中文轉英文）
translator = Translator()

def translate_to_english(text):
    try:
        result = translator.translate(text, dest='en')
        return result.text
    except Exception as e:
        print(f"[⚠️ 翻譯失敗] {text} -> {e}")
        return ""

# ✅ Step 4: 載入 A/B 表資料
vendor_A_table = f"{project_id}.{dataset}.vendor_A"
vendor_B_table = f"{project_id}.{dataset}.vendor_B"

query_A = f"SELECT product_id, product_name, product_location_country, price FROM `{vendor_A_table}`"
query_B = f"SELECT product_id, product_name, product_location_country, price FROM `{vendor_B_table}`"

df_A = client.query(query_A).to_dataframe()
df_B = client.query(query_B).to_dataframe()

# ✅ Step 5: 翻譯商品名稱
print("🔤 正在翻譯商品名稱...")
df_A['product_name_en'] = df_A['product_name'].apply(translate_to_english)
df_B['product_name_en'] = df_B['product_name'].apply(translate_to_english)

# ✅ Step 6: 英文斷詞 function（空格切詞）
def tokenize(text):
    if not isinstance(text, str):
        return set()
    return set(text.lower().split())

# ✅ Step 7: 比對 function（多核心）
def compare_product(row_a):
    row_results = []
    tokens_a = tokenize(row_a['product_name_en'])
    country = row_a['product_location_country']
    df_b_filtered = df_B[df_B['product_location_country'] == country]

    best_score = 0
    best_b_row = None

    for _, row_b in df_b_filtered.iterrows():
        tokens_b = tokenize(row_b['product_name_en'])

        if abs(len(tokens_a) - len(tokens_b)) > 5:
            continue

        intersection = tokens_a & tokens_b
        union = tokens_a | tokens_b
        jaccard_score = len(intersection) / len(union) if union else 0

        if jaccard_score > best_score:
            best_score = jaccard_score
            best_b_row = row_b

    if best_b_row is not None and best_score >= 0.2:
        vendor_a_price = row_a['price'] if row_a['price'] is not None else 0
        vendor_b_price = best_b_row['price'] if best_b_row['price'] is not None else 0
        price_diff = vendor_b_price - vendor_a_price

        row_results.append({
            'product_location_country': country,
            'vendor_A_product_id': row_a['product_id'],
            'vendor_A_product_name': row_a['product_name'],
            'vendor_A_product_name_en': row_a['product_name_en'],
            'vendor_A_price': vendor_a_price,
            'vendor_B_product_id': best_b_row['product_id'],
            'vendor_B_product_name': best_b_row['product_name'],
            'vendor_B_product_name_en': best_b_row['product_name_en'],
            'vendor_B_price': vendor_b_price,
            'jaccard_score': best_score,
            'price_diff': price_diff
        })

    return row_results

# ✅ Step 8: 多進程比對執行
print("🚀 啟動多核心比對中...")

with Pool(processes=os.cpu_count()) as pool:
    all_results = list(tqdm(pool.imap(compare_product, df_A.to_dict(orient='records')), total=len(df_A)))

# ✅ Step 9: 合併結果
flat_results = list(itertools.chain.from_iterable(all_results))
matched_df = pd.DataFrame(flat_results)

# ✅ Step 10: 寫入 BigQuery
today_str = datetime.today().strftime('%Y%m%d')
destination_table = f"{dataset}.experience_products_matched_results_en_{today_str}"

matched_df.to_gbq(destination_table=destination_table,
                  project_id=project_id,
                  if_exists='replace')

print(f"✅ 共比對成功 {len(matched_df)} 筆資料")
print(f"✅ 已寫入 BigQuery 表格：{destination_table}")



🔤 正在翻譯商品名稱...
🚀 啟動多核心比對中...


100%|██████████| 3473/3473 [03:42<00:00, 15.59it/s]
/tmp/ipython-input-1-749729985.py:116: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  matched_df.to_gbq(destination_table=destination_table,
100%|██████████| 1/1 [00:00<00:00, 4198.50it/s]

✅ 共比對成功 2641 筆資料
✅ 已寫入 BigQuery 表格：jessica_test.experience_products_matched_results_en_20250724
